# Scrape a GIS file of South Dakota roadkill

We want to get the GIS data behind [this map of wildlife roadkill reported in South Dakota](https://experience.arcgis.com/experience/58b6d830b554429b84468e9b0feb947a/) and load it into a geodataframe. To do that, we're going to use [a Python library called `esridump`](https://pypi.org/project/esridump/) to fetch the GIS data from an ESRI server and return it as a list of geojson features.

First, we need to find the URL of the actual ESRI server hosting the data. Here's how to do that.

Navigate to the map page and open your browser's network inspection tools:
- Chrome: View > Developer > Developer Tools
- Firefox: Tools > Browser Tools > Web Developer Tools

The keyboard shortcut for both should be Command + Option + I (on a Mac) or Control + Option + I (on a PC).

In the developer console, flip over to the `Network` tab. Reload the page and watch the requests sent to the target server. Look for a request to a URL that includes the word `query` (there will be several).

Select that network request from the list and look at the `Headers` section, which will show you the URL to which the browser just sent a `GET` request. Mine looks like this:

```
https://services1.arcgis.com/PwrabBhZHUggYYSp/arcgis/rest/services/survey123_2fa2f046e8e0422fb99f2836d19beb52_stakeholder/FeatureServer/1/query?f=pbf&maxRecordCountFactor=4&resultOffset=32000&resultRecordCount=4000&where=(Latitude IS NOT NULL) AND (Species <> 'No Pick-Up') AND (Found = 'Yes')&orderByFields=objectid&outFields=objectid&outSR=102100&spatialRel=esriSpatialRelIntersects
```

We don't want the whole URL, just the part before `query`:

```
https://services1.arcgis.com/PwrabBhZHUggYYSp/arcgis/rest/services/survey123_2fa2f046e8e0422fb99f2836d19beb52_stakeholder/FeatureServer/1/
```

[Navigate to that URL](https://services1.arcgis.com/PwrabBhZHUggYYSp/arcgis/rest/services/survey123_2fa2f046e8e0422fb99f2836d19beb52_stakeholder/FeatureServer/1/) and you'll see the underlying ESRI server. This is the URL we'll hand off to the `esridump` tool.

In [ ]:
from esridump.dumper import EsriDumper
import geopandas as gpd

In [ ]:
target_url = 'https://services1.arcgis.com/PwrabBhZHUggYYSp/arcgis/rest/services/survey123_2fa2f046e8e0422fb99f2836d19beb52_stakeholder/FeatureServer/1/'

# grab the list of features as a list
feature_data = list(EsriDumper(target_url))

In [ ]:
# print a message to let us know how we did
print(f'Fetched {len(feature_data):,} features.')

In [ ]:
# build a valid geojson object with the data
# https://geojson.org/
geojson = {
    'type': 'FeatureCollection',
    'features': feature_data
}

In [ ]:
gdf = gpd.GeoDataFrame.from_features(geojson)

In [ ]:
gdf.head()

In [ ]:
gdf.plot()

In [ ]:
gdf.to_file('south-dakota-roadkill.geojson', driver='GeoJSON')